In [1]:
from sorf_query import *
import jsonlines
import pandas as pd

In [2]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
pd.options.display.max_colwidth = 200

In [3]:
GENOME_REFERENCE_PATH = 's3://velia-annotation-dev/genomes/hg38/GRCh38.p13.genome.fa.gz'

In [4]:
with open('test_vtx.txt') as fhandle:
    ids = [int(i.replace('VTX-', '')) for i in fhandle.readlines()]
ids[:5]

[7082, 7146, 7396, 7541, 9335]

In [5]:
new_ids = [851390, 81996, 850636]
ids = new_ids+ids

In [6]:
# session.rollback()

In [194]:
find_seq_substring('ATGAGACGCTTCGACTGTCTCATCGGGGCACTTGTAATAAGCATCTTGGTGCCACTGAATGCAATGCTGTATTCAAATAATAGCTTTCATCTTCACTCTTTT',
                   transcripts)

['ENST00000665868.2|ENSG00000229989.6|OTTHUMG00000035888.5|OTTHUMT00000517878.1|MIR181A1HG-206|MIR181A1HG|3209|lncRNA|',
 'ENST00000436880.2|ENSG00000229989.6|OTTHUMG00000035888.5|OTTHUMT00000087436.2|MIR181A1HG-203|MIR181A1HG|2308|lncRNA|']

In [192]:
[i for i in orfs if i.id == 786168][0].nt_seq

'ATGATATTGCAGCTTTTTCTTTTGGTTGCGTGCAGTGAGAATCTGGGAGCTGAACCTGTTATCTGCATGGTCTTCAGAAATCAGGCAAACTCGGAAAATGCCAACGCCAAAAATGCTGATGGG'

In [7]:
session = base.Session()

In [8]:
orfs = session.query(Orf).filter(Orf.id.in_(ids)).all()
current_orf = orfs[0]
missing_orfs = set(ids) - set([i.id for i in orfs])
missing_orfs

set()

In [9]:
transcript_matching_results = run_id_mapping_parallel(orfs[:8], NCPU = 1)

  0%|          | 0/8 [00:00<?, ?it/s]/home/ec2-user/anaconda/envs/veliadash/lib/python3.11/site-packages/Bio/Seq.py:2804: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(
100%|██████████| 8/8 [00:30<00:00,  3.87s/it]


In [36]:
import numpy as np

In [167]:
[i for i in np.where(np.array([i.chrom_starts.count(';') for i in orfs])>0)[0] if orfs[i].strand=='-']

[259,
 262,
 265,
 271,
 297,
 298,
 310,
 321,
 322,
 333,
 334,
 335,
 343,
 344,
 345,
 346,
 374,
 375,
 376,
 377,
 378,
 379,
 390,
 391,
 393,
 397,
 398,
 399,
 400,
 423,
 424,
 435,
 443,
 444,
 445,
 446,
 453,
 454,
 464,
 465,
 481,
 483,
 486,
 489,
 494,
 496,
 500,
 505,
 507,
 508,
 516,
 521,
 528,
 530,
 532,
 537,
 541,
 542,
 544,
 546,
 547,
 559,
 561,
 573,
 578,
 579,
 580,
 593,
 595,
 596,
 597,
 601,
 604,
 605,
 606,
 611,
 615,
 619,
 621,
 624,
 625,
 626,
 628,
 632,
 633,
 636,
 638,
 640]

In [40]:
orfs[259]

ORF: 849976, VTX-0849976, CXCL2_74964836_75aa,(-)chr4:74098795-74099120

In [121]:
reference[chrom][0:3]

>chr4:1-3
NNN

In [114]:
reference[chrom][chrom_starts[i]:(chrom_starts[i] + blocks[i])].complement.seq[::-1]

'GAGCGCCCCTGGCCACTGAACTGCGCTGCCAGTGCTTGCAGACCCTGCAGGGAATTCACCTCAAGAACATCCAAAGTGTGAAGGTGAAGTCCCCCGGACCCCACTGCGCCCAAACCGAAGTCATG'

<module 'pyfaidx' from '/home/ec2-user/anaconda/envs/veliadash/lib/python3.11/site-packages/pyfaidx/__init__.py'>

In [185]:

        o = orfs[259]
        blocks = [int(x) for x in o.block_sizes.split(';')]
        chrom_starts = [int(x) - 1 for x in o.chrom_starts.split(';')]
        start = o.start - 1
        strand = o.strand
        chrom = o.assembly.ucsc_style_name
        seqs = []
        for i, block in enumerate(blocks):
            if i == len(blocks)-1 and strand == '+':
                blocks[i] = blocks[i] - 3
                
            elif i == 0 and strand == '-':
                blocks[i] = blocks[i] + 3
                
            if strand == '-':
                seqs.append(reference[chrom][chrom_starts[i]:(chrom_starts[i] + blocks[i])].complement.seq[::-1].upper())
            else:
                seqs.append(reference[chrom][chrom_starts[i]:(chrom_starts[i] + blocks[i])].seq.upper())
        
        if strand == '-':
            seqs.reverse()
        seq = ''.join(seqs)
        strand

'-'

In [187]:
o.nt_seq == seq

True

In [188]:
seqs

['ATGGCCCGCGCCACGCTCTCCGCCGCCCCCAGCAATCCCCGGCTCCTGCGGGTGGCGCTGCTGCTCCTGCTCCTGGTGGCCGCCAGCCGGCGCGCAGCAG',
 'GAGCGCCCCTGGCCACTGAACTGCGCTGCCAGTGCTTGCAGACCCTGCAGGGAATTCACCTCAAGAACATCCAAAGTGTGAAGGTGAAGTCCCCCGGACCCCACTGCGCCCAAACCGAAGTCATG']

In [189]:
o.aa_seq

'MARATLSAAPSNPRLLRVALLLLLLVAASRRAAGAPLATELRCQCLQTLQGIHLKNIQSVKVKSPGPHCAQTEVM'

In [172]:
    orf = orfs[444]
    # if orf.start == -1 or orf.end == -1:
        # return None
    chrom = orf.assembly.ucsc_style_name
    blocks = list(zip(orf.chrom_starts.split(';'), orf.block_sizes.split(';'), orf.phases.split(';')))
    strand = orf.strand
    nucl = []
    for ix, (start, size, phase) in enumerate(blocks):
        start = int(start)-1
        size = int(size)
        phase = int(phase)
        if strand == '-' and ix == 0:
            # start = start-3
            size += 3
        if strand == '-':
            s = reference[chrom][start : start+size]
            s = s.complement.seq[::-1]
        else:
            s = reference[chrom][start : start+size]
            s = s.seq
        nucl.append(s)
    # if strand == '-' and len(blocks)>1:
    #     nucl[-1] = reference[chrom][start+3 : start+size].complement.seq[::-1]
    # elif strand == '-' and len(blocks)==1:
    #     nucl[0] = nucl[0][1:]
    if strand == '-':
        nucl = nucl[::-1]
    #     nucl = ' '.join(nucl)
    # else:
    #     nucl = ' '.join(nucl)

In [173]:
orf.aa_seq

'VVESAIHTLVPLFFLAVVVTQTEGTTTEVECPTEGTTTRTSEDEETIVATKINLRATTSGSRV'

In [174]:
nucl

['GTGGTGGAATCGGCTATCCATACCCTCGTGCCCCTGTTTTTCCTGGCCGTGGTAGTTACTCAAACAGAGGGAACTACAACAGAGGTGGAATGCCCAACAGAGGGAACTACAACCAG',
 'AACTTCAGAGGACGAGGAAACAATCGTGGCTACAAAAATCAATCTCAGGGCTACAACCAGTGGCAGCAGGGTG']

In [100]:
chrom, blocks, strand

('chr4', [('74098798', '122', '2'), ('74099021', '100', '0')], '-')

In [53]:
def create_comparison_groups_xena_tcga_vs_normal(xena):
    groups = {}
    for ix, row in tqdm(tissue_pairs.iterrows()):
        # Normal sample indexes for specific tissues
        normal = xena.index[(xena['_primary_site'].str.lower() == row['GTEx Tissue Type'].lower())
                                & (xena['_sample_type'] == 'Normal Tissue')]
        # Cancer sample indexes
        cancer = xena.index[(xena['detailed_category'].str.lower() == row['Description'].lower())
                                & (xena['_sample_type'] == 'Primary Tumor')]
        groups[row['TCGA Cancer Type']] = {'normal_indices': normal, 'cancer_indices': cancer,
                                       'GTEx Tissue': row['GTEx Tissue Type'], 'TCGA Cancer': row['Description']}
    return groups

In [61]:
import numpy as np

In [67]:
r

NameError: name 'r' is not defined

In [71]:
xena.loc[g['normal_indices']][xena.columns[6:]].mean(axis=0)

ENST00000302101   -3.248698
ENST00000299415   -8.989675
ENST00000596676   -7.941365
ENST00000527149    1.095069
ENST00000525141    4.498505
ENST00000326842   -9.749581
ENST00000301200    2.436686
ENST00000298966    2.714475
dtype: float64

In [83]:
'ENST00000253122' in xena.columns

False

In [88]:
xena = pd.read_parquet('../../.cache/xena.parq')
tissue_pairs = pd.read_parquet('../../.cache/gtex_tcga_pairs.parq')
normal_vs_gtex_expression = pd.read_parquet('../../.cache/gtex_tcga_pairs.parq')
normal_vs_gtex_expression.index = normal_vs_gtex_expression['Transcript']
normal_vs_gtex_expression.loc['ENST00000253122']

,Transcript,TCGA,GTEx,Description,Normal,Cancer
Transcript,,,,,,
ENST00000253122,ENST00000253122,ACC,adrenal gland,Adrenocortical Cancer,33.600359,49.264961
ENST00000253122,ENST00000253122,BLCA,bladder,bladder urothelial carcinoma,37.728066,40.941452
ENST00000253122,ENST00000253122,BRCA,breast,breast invasive carcinoma,16.239323,14.747659
ENST00000253122,ENST00000253122,CESC,cervix uteri,Cervical & Endocervical Cancer,15.126069,51.724043
ENST00000253122,ENST00000253122,CHOL,liver,cholangiocarcinoma,1.640086,16.829578
ENST00000253122,ENST00000253122,COAD,colon,colon adenocarcinoma,45.299947,24.193626
ENST00000253122,ENST00000253122,DLBC,blood,Diffuse Large B-Cell Lymphoma,4.209813,1.067385
ENST00000253122,ENST00000253122,ESCA,esophagus,esophageal carcinoma,31.661973,46.371972
ENST00000253122,ENST00000253122,GBM,brain,glioblastoma multiforme,38.757408,57.155692


In [ ]:
cancer = xena.index[(xena['detailed_category'].str.lower() == row['Description'].lower())
                                & (xena['_sample_type'] == 'Primary Tumor')]

In [81]:
normal_vs_gtex_expression

,Transcript,Cancer,GTEx,Description,Normal,Cancer
0,ENST00000302101,ACC,adrenal gland,Adrenocortical Cancer,-3.248698,-4.078971
1,ENST00000299415,ACC,adrenal gland,Adrenocortical Cancer,-8.989675,-8.215799
2,ENST00000596676,ACC,adrenal gland,Adrenocortical Cancer,-7.941365,-8.554186
3,ENST00000527149,ACC,adrenal gland,Adrenocortical Cancer,1.095069,-0.681665
4,ENST00000525141,ACC,adrenal gland,Adrenocortical Cancer,4.498505,4.025062
...,...,...,...,...,...,...
243,ENST00000527149,UVM,skin,uveal melanoma,-5.015234,-4.888420
244,ENST00000525141,UVM,skin,uveal melanoma,2.904968,1.755980
245,ENST00000326842,UVM,skin,uveal melanoma,-9.390312,-8.936177
246,ENST00000301200,UVM,skin,uveal melanoma,3.299456,3.419008


In [80]:
normal_vs_gtex_expression[normal_vs_gtex_expression['Transcript'] == 'ENST00000253122']

,Transcript,Cancer,GTEx,Description,Normal,Cancer


In [33]:
metadata = pd.read_table('s3://velia-data-dev/VDC_004_annotation/20230329_tcga_data/Xena/TcgaTargetGTEX_phenotype.txt')
metadata

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xca in position 15827: invalid continuation byte

Index(['GTEX-S4Q7-0003-SM-3NM8M', 'TCGA-19-1787-01', 'TCGA-S9-A7J2-01',
       'GTEX-QV31-1626-SM-2S1QC', 'TCGA-G3-A3CH-11',
       'GTEX-13OVI-1026-SM-5L3EM', 'GTEX-13OW5-0626-SM-5J2N2',
       'GTEX-XUZC-2026-SM-4BRW9', 'TCGA-EK-A2RE-01', 'TCGA-D5-5538-01',
       ...
       'TCGA-IB-7885-01', 'TCGA-B6-A0IA-01', 'GTEX-1117F-2826-SM-5GZXL',
       'TCGA-VQ-AA6F-01', 'TCGA-BR-8588-01', 'GTEX-11ZTS-3326-SM-5LU9Y',
       'TCGA-DD-A115-01', 'GTEX-ZA64-2126-SM-5Q5A8', 'TCGA-FV-A3I0-11',
       'GTEX-XV7Q-0426-SM-4BRVN'],
      dtype='object', length=19126)

In [39]:
    transcripts_to_load = transcript_matching_results
    
    xena_transcripts = pd.read_csv('s3://velia-data-dev/VDC_004_annotation/20230329_tcga_data/xena_transcript_names.csv')
    if transcripts_to_load is None:
        transcripts_to_map = xena_transcripts['Xena Transcripts']
    else:
        overlapping_xena_sorf_transcripts = set(xena_transcripts['Xena Transcripts']).intersection(set(transcripts_to_load))
    xena = pd.read_feather('s3://velia-data-dev/VDC_004_annotation/20230329_tcga_data/xena.feather', columns = ['index']+list(overlapping_xena_sorf_transcripts))
    xena.index = xena.pop('index')
    metadata = pd.read_table('s3://velia-data-dev/VDC_004_annotation/20230329_tcga_data/Xena/TcgaTargetGTEX_phenotype.txt',
                             encoding = "ISO-8859-1", index_col=0)
    metadata = metadata[~metadata['_primary_site'].isna()]
    ordr = xena.index.intersection(metadata.index)
    xena = xena.loc[ordr].copy()
    metadata = metadata.loc[ordr].copy()

    tissue_pairs = pd.read_excel('s3://velia-data-dev/VDC_004_annotation/20230329_tcga_data/Xena/GTEx_TCGA_comparisons.xlsx')
    tissue_pairs['GTEx Tissue Type'] = tissue_pairs['GTEx Tissue Type'].str.replace('whole ', '')
    tissue_pairs = tissue_pairs[tissue_pairs['GTEx Tissue Type']!='--']
    
    # tissue_pairs.head(4)

,TCGA Cancer Type,Description,GTEx Tissue Type,Ambiguity,Unnamed: 4,Unnamed: 5
0,ACC,Adrenocortical Cancer,adrenal gland,N,NaN,NaN
1,BLCA,bladder urothelial carcinoma,bladder,N,NaN,NaN
2,BRCA,breast invasive carcinoma,breast,N,NaN,NaN
3,CESC,Cervical & Endocervical Cancer,cervix uteri,N,NaN,NaN


In [35]:
metadata

,detailed_category,primary disease or tissue,_primary_site,_sample_type,_gender,_study
sample,,,,,,
TCGA-V4-A9EE-01,Uveal Melanoma,Uveal Melanoma,Eye,Primary Tumor,Male,TCGA
TCGA-VD-AA8N-01,Uveal Melanoma,Uveal Melanoma,Eye,Primary Tumor,Male,TCGA
TCGA-V4-A9EI-01,Uveal Melanoma,Uveal Melanoma,Eye,Primary Tumor,Male,TCGA
TCGA-VD-AA8O-01,Uveal Melanoma,Uveal Melanoma,Eye,Primary Tumor,Male,TCGA
TCGA-WC-A888-01,Uveal Melanoma,Uveal Melanoma,Eye,Primary Tumor,Male,TCGA
...,...,...,...,...,...,...
TARGET-20-PANPKN-09,Acute Myeloid Leukemia,Acute Myeloid Leukemia,White blood cell,Primary Blood Derived Cancer - Bone Marrow,Male,TARGET
TARGET-20-PANLIR-09,Acute Myeloid Leukemia,Acute Myeloid Leukemia,White blood cell,Primary Blood Derived Cancer - Bone Marrow,Male,TARGET
TARGET-20-PAPAWN-09,Acute Myeloid Leukemia,Acute Myeloid Leukemia,White blood cell,Primary Blood Derived Cancer - Bone Marrow,Male,TARGET


In [21]:
transcript_matching_results[7082][-1]

['ENST00000525141', 'ENST00000298966', 'ENST00000527149', 'ENST00000596676']

In [22]:
overlapping_tids

['XM_017023741.2', 'ENST00000329565.6']

In [25]:
transcripts_to_map = []
with jsonlines.open('sorf_table.jsonlines', mode = 'w') as fh:
    for current_orf in tqdm(orfs):
        # nt, aa = extract_nucleotide_sequence_broken_psl_starts(current_orf, reference)
        overlapping_tids = query_overlapping_transcripts(current_orf, session)
        overlapping_tids = [[i.split('.')[0] for i in [t.ensembl_id, t.refseq_id, t.chess_id] if i][0] for t in overlapping_tids]
        # exact_tids = find_seq_substring(nt, transcripts)
        # if len(tids)>0:
        # exact_tids = [i.split('|')[0].split('.')[0] for i in exact_tids]
        exact_tids = transcript_matching_results[current_orf.id][-1]
        nt = transcript_matching_results[current_orf.id][0]
        aa = transcript_matching_results[current_orf.id][1]
        attributes = {
            'chr': current_orf.assembly.ucsc_style_name,
            'vtx': f"VTX-{str(current_orf.id).zfill(7)}",
            'strand': current_orf.strand,
            'start': current_orf.start,
            'end': current_orf.end,
            'nucl': nt,
            'aa': aa,
            'transcripts_exact': exact_tids,
            'transcripts_overlapping': overlapping_tids,
            
        }
        transcripts_to_map+=exact_tids
        transcripts_to_map+=[i.split('.')[0] for i in overlapping_tids if i.startswith('ENST')]
        
        fh.write(attributes)
        # break

  1%|          | 8/644 [00:00<00:49, 12.85it/s]


KeyError: 11192

In [ ]:
assembly_ids = {}

for assembly in session.query(base.Assembly).all():
    if assembly.ucsc_style_name.startswith('chr') and len(assembly.ucsc_style_name) < 6:
        assembly_ids[assembly.ucsc_style_name] = assembly
    else:
        assembly_ids[assembly.genbank_accession] = assembly

In [ ]:
assembly_ids

In [17]:
from Bio import SeqIO

In [21]:
with smart_open.open('s3://velia-annotation-dev/gencode/v42/gencode.v42.transcripts.fa.gz') as f:
    seqs = SeqIO.to_dict(SeqIO.parse(f, 'fasta'))

In [24]:
# seqs.keys()

# Map transcripts

In [253]:
targets = {
    'VTX-0851390': 'ATGGCCGAGAGGCCGGGGCCTCCGGGCGGCGCCGTGTCCGCGACCGCGTACCCTGACACCCCCGCGGAATTCCCTCCGCACCTCCAGGCGGGTGCGATGCGGCGCCGCTTTTGGGGCGTATTCAACTGTCTGTGCGCCGGCGCGTTCGGGGCCCTGGCCGCCGCCTCCGCCAAGCTGGCCTTCGGCAGCGAGGTGAGCATGGGTTTATGCGTCTTAGGCATTATTGTGATGGCGAGCACCAATTCTCTGATGTGGACCTTCTTTAGCCGGGGCCTCAGTTTCTCCATGTCTTCAGCCATTGCATCTGTCACAGTGACTTTTTCAAATATCCTCAGCTCGGCCTTCCTGGGCTATGTGCTGTATGGAGAGTGCCAGGAGGTCTTGTGGTGGGGAGGAGTGTTCCTTATTCTCTGCGGACTCACCCTAATCCACAGGAAGCTCCCACCCACCTGGAAGCCCCTTCCACACAAGCAGCAG',
    'VTX-0081996': 'ATGGCGGATGTGTCAGAGAGGACACTGCAGTTGTCCGTGCTAGTAGCCTTCGCTTCTGGAGTACTCCTGGGCTGGCAGGCGAACCGACTGCGGAGGCGCTACTTGGACTGGAGGAAAAGGAGGCTGCAGGACAAGCTGGCGGCGACGCAGAAGAAGCTGGACCTGGCC',
    'VTX-0087927': 'ATGAACGGCTCTCAGGCGGGCGCCGCGGCTCAGGCCGCCTGGCTGAGCTCCTGCTGTAACCAGTCGGCGTCGCCGCCGGAGCCCCCCGAGGGGCCGCGCGCGGTGCAGGCGGTGGTGCTCGGCGTGCTGTCCCTGCTGGTGCTTTGCGGGGTCCTGTTCCTGGGCGGCGGCCTCCTCCTCCGCGCCCAGGGCCTGACAGCGCTGCTGACCCGCGAGCAGCGCGCGTCCCGCGAGCCCGAGCCGGGCAGTGCCAGCGGAGAGGACGGCGACGACGACTCC',
    'VTX-0085840': 'ATGCTCCTGGGGAGTCTGTGGGGAAGATGCCATCCAGGGCGCTGTGCGCTCTTCCTCATCCTCGCCCTCCTGCTGGACGCGGTCGGCCTGGTCCTTTTGCTGCTGGGGATCTTGGCCCCCCTGAGTTCCTGGGACTTCTTCATCTACACAGGTGCCCTGATCCTGGCTCTCAGCCTACTGCTCTGGATCATCTGGTATTCCCTCAACATTGAGGTGTCTCCTGAAAAACTGGACCTG',
    'VTX-0087707': 'ATGACCTCCTGGCCAGGTGGCAGCTTTGGCCCTGACCCGCTCCTGGCCCTGCTGGTGGTGATCCTGCTAGCACGCCTCATCCTGTGGTCCTGCCTCGGGACCTACATCGACTACAGACTGGCCCAGCGGCGGCCCCAGAAACCCAAGCAGGAC',
}
for i, s in targets.items():
    hit = find_seq_substring(s, transcripts)
    print(i, hit)

VTX-0851390 ['ENST00000302392.5|ENSG00000169964.8|OTTHUMG00000133092.3|OTTHUMT00000256750.3|TMEM42-201|TMEM42|977|protein_coding|']
VTX-0081996 ['ENST00000611969.5|ENSG00000175701.11|OTTHUMG00000131194.5|OTTHUMT00000477505.2|MTLN-203|MTLN|427|protein_coding|', 'ENST00000414416.2|ENSG00000175701.11|OTTHUMG00000131194.5|OTTHUMT00000253918.4|MTLN-201|MTLN|2051|protein_coding|', 'ENST00000426713.1|ENSG00000175701.11|OTTHUMG00000131194.5|OTTHUMT00000338107.3|MTLN-202|MTLN|461|protein_coding|']
VTX-0087927 ['ENST00000546390.2|ENSG00000284791.2|OTTHUMG00000169615.4|OTTHUMT00000405071.2|SMIM41-201|SMIM41|1430|protein_coding|']
VTX-0085840 ['ENST00000581851.2|ENSG00000263429.4|OTTHUMG00000132847.7|OTTHUMT00000256321.4|TMEM238L-202|TMEM238L|1535|protein_coding|']
VTX-0087707 ['ENST00000641568.1|ENSG00000284713.2|OTTHUMG00000172974.2|OTTHUMT00000491249.1|SMIM38-202|SMIM38|2695|protein_coding|', 'ENST00000686937.1|ENSG00000284713.2|OTTHUMG00000172974.2|-|SMIM38-204|SMIM38|5447|protein_coding|', 'E

In [251]:
import gzip
import fsspec
from pyfaidx import Fasta
import pandas as pd
from tqdm import tqdm

# deduped_conservation = pd.read_parquet('deduped_phylocsf_length_filtered.parq')
transcripts = Fasta('./gencode.v42.transcripts.fa')

def find_seq_substring(query, target_dict):
    return [k for k, s in target_dict.items() if query.lower() in s[:].seq.lower()]

def extract_sequence(reference, chrom, strand, block_starts, block_sizes):
    s = reference[chrom]
    sequence = ''
    for start, length in zip(block_starts, block_sizes):
        end = start + (3*(length-1)) + 3
        sequence += str(s[start : end])
    sequence+=str(s[end:end+3])
    return sequence

def pfunc(row):
    ix = row['qName']
    starts = tuple(map(int, row['tStarts'].strip(',').split(',')))
    sizes = tuple(map(int, row['blockSizes'].strip(',').split(',')))
    s = extract_sequence(genes, row['tName'], row['strand'], starts, sizes)
    # print(row['strand'], s[:3], s[-3:])
    # blat_sequences[row['qName']] = s
    matches = find_seq_substring(s, transcripts)
    if len(matches)>0:
        transcript_names = [i.split('|')[0].split('.')[0] for i in matches]
        gene_names = [i.split('|')[5] for i in matches]
        return ix, {'transcripts': transcript_names, 'genes': gene_names, 'sequence': s}
    else:
        return ix, {'transcripts': [], 'genes': [], 'sequence': s}
    
# import multiprocessing as mp
# with mp.Pool(120) as ppool:
#     annotations = {}
#     ixes, rows = list(zip(*deduped_conservation.iterrows()))
#     for r in tqdm(ppool.imap(pfunc, rows), total=50000):#, total=signalp_and_conserved.shape[0]):
#         annotations[r[0]] = r[1]